# Cài đặt thư viện

In [12]:
!pip install requests beautifulsoup4
!pip install pymongo

# Lấy URL

In [13]:
import requests
from bs4 import BeautifulSoup
import csv
import pandas as pd

# URL của trang web
url = 'https://xebuyt.net/tuyen-xe-buyt'

# Gửi request để lấy nội dung trang
response = requests.get(url)
response.raise_for_status()  # Kiểm tra lỗi nếu có

# Parse HTML với BeautifulSoup
soup = BeautifulSoup(response.content, 'html.parser')

# Tìm div chứa danh sách tuyến
div_result = soup.find('div', id='divResult')

# Tìm tất cả các thẻ <a> với class 'cms-button'
route_links = div_result.find_all('a', class_='cms-button')

# Danh sách để lưu dữ liệu
data = []

# Duyệt qua từng link
for link in route_links:
    # Lấy href
    href = link.get('href')
    full_url = 'https://xebuyt.net' + href if href.startswith('/') else href

    # Lấy số tuyến từ span.icon
    route_number = link.find('span', class_='icon').get_text(strip=True)

    # Lấy tên tuyến từ div.routetrip
    route_name = link.find('div', class_='routetrip').get_text(strip=True)

    # Thêm vào danh sách
    data.append({
        'Route_Number': route_number,
        'Route_Name': route_name,
        'URL': full_url
    })

# Lưu vào file CSV
filename = 'tuyen_xe_buyt.csv'
with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['Route_Number', 'Route_Name', 'URL']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    writer.writerows(data)

print(f'Dữ liệu đã được lưu vào file {filename}. Tổng số tuyến: {len(data)}')

# Hiển thị dưới dạng DataFrame (comment nếu không cần)
df = pd.DataFrame(data)
df.head()

Dữ liệu đã được lưu vào file tuyen_xe_buyt.csv. Tổng số tuyến: 140


,Route_Number,Route_Name,URL
0,01,Bến Thành- BX Chợ Lớn,https://xebuyt.net/ben-thanh-ben-xe-cho-lon.html
1,02,Bến Thành- BX Miền Tây,https://xebuyt.net/ben-thanh-ben-xe-mien-tay.html
2,03,Bến Thành- Thạnh Lộc,https://xebuyt.net/ben-thanh-thanh-loc.html
3,04,Bến Thành- Cộng Hòa- An Sương,https://xebuyt.net/ben-thanh-cong-hoa-an-suong.html
4,05,Bến xe Chợ Lớn - Biên Hòa,https://xebuyt.net/ben-xe-cho-lon-bien-hoa.html


# Lấy thông tin chuyến xe

In [14]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import re

# Đọc file CSV đã tạo trước đó
csv_filename = 'tuyen_xe_buyt.csv'
df = pd.read_csv(csv_filename)

# Danh sách để lưu dữ liệu mở rộng
extended_data = []

# Headers cho CSV mới
headers = [
    'Route_Number', 'Route_Name', 'URL',
    'Operator', 'Operating_Type', 'Distance', 'Vehicle_Type', 'Operating_Hours',
    'Ticket_Price', 'Number_of_Trips', 'Trip_Time', 'Frequency'
]

# Duyệt qua từng hàng trong DataFrame
for index, row in df.iterrows():
    route_number = row['Route_Number']
    route_name = row['Route_Name']
    url = row['URL']

    print(f"Đang scrape tuyến {route_number}: {route_name}")

    try:
        # Gửi request
        response = requests.get(url)
        response.raise_for_status()

        # Parse HTML
        soup = BeautifulSoup(response.content, 'html.parser')

        # Tìm div rouInfo và table
        rou_info_div = soup.find('div', id='rouInfo')
        if not rou_info_div:
            print(f"Không tìm thấy div#rouInfo cho tuyến {route_number}")
            continue

        table = rou_info_div.find('table', class_='tbl100')
        if not table:
            print(f"Không tìm thấy table cho tuyến {route_number}")
            continue

        # Khởi tạo giá trị mặc định
        operator = ''
        operating_type = ''
        distance = ''
        vehicle_type = ''
        operating_hours = ''
        ticket_price = ''
        number_of_trips = ''
        trip_time = ''
        frequency = ''

        # Duyệt qua các hàng trong table
        rows = table.find('tbody').find_all('tr')
        for tr in rows:
            tds = tr.find_all('td')
            if len(tds) < 2:
                continue

            label_td = tds[0].get_text(strip=True)
            content = tds[1].get_text(separator='\n').strip()

            # Xử lý Đơn vị đảm nhận (có thể ở hàng riêng)
            if 'Đơn vị đảm nhận' in label_td:
                operator = content

            # Bỏ qua hàng spacing
            if not label_td and not content:
                continue

            # Xử lý ul lists ở hàng cuối (cột 2 và 3)
            if len(tds) >= 3 and not label_td:  # Hàng có ul lists
                left_ul = tds[1].find('ul')
                right_ul = tds[2].find('ul') if len(tds) > 2 else None

                if left_ul:
                    left_items = [li.get_text(strip=True) for li in left_ul.find_all('li')]
                    for item in left_items:
                        if 'Loại hình hoạt động' in item:
                            operating_type = item.replace('Loại hình hoạt động:', '').strip()
                        elif 'Cự ly' in item:
                            distance = item.replace('Cự ly:', '').strip()
                        elif 'Loại xe' in item:
                            vehicle_type = item.replace('Loại xe:', '').strip()
                        elif 'Thời gian hoạt động' in item:
                            operating_hours = item.replace('Thời gian hoạt động:', '').strip()

                if right_ul:
                    right_items = [li.get_text(separator='\n').strip() for li in right_ul.find_all('li')]
                    for item in right_items:
                        if 'Giá vé' in item:
                            ticket_price = item.replace('Giá vé:', '').strip()
                        elif 'Số chuyến' in item:
                            number_of_trips = item.replace('Số chuyến:', '').strip()
                        elif 'Thời gian chuyến' in item:
                            trip_time = item.replace('Thời gian chuyến:', '').strip()
                        elif 'Giãn cách chuyến' in item:
                            frequency = item.replace('Giãn cách chuyến:', '').strip()

        # Thêm vào danh sách
        extended_data.append({
            'Route_Number': route_number,
            'Route_Name': route_name,
            'URL': url,
            'Operator': operator,
            'Operating_Type': operating_type,
            'Distance': distance,
            'Vehicle_Type': vehicle_type,
            'Operating_Hours': operating_hours,
            'Ticket_Price': ticket_price,
            'Number_of_Trips': number_of_trips,
            'Trip_Time': trip_time,
            'Frequency': frequency
        })

    except Exception as e:
        print(f"Lỗi khi scrape tuyến {route_number}: {e}")
        # Vẫn thêm dữ liệu với giá trị rỗng
        extended_data.append({
            'Route_Number': route_number,
            'Route_Name': route_name,
            'URL': url,
            'Operator': '',
            'Operating_Type': '',
            'Distance': '',
            'Vehicle_Type': '',
            'Operating_Hours': '',
            'Ticket_Price': '',
            'Number_of_Trips': '',
            'Trip_Time': '',
            'Frequency': ''
        })

    # Delay để tránh overload server
    time.sleep(1)

# Tạo DataFrame và lưu vào CSV mới
extended_df = pd.DataFrame(extended_data)
output_filename = 'tuyen_xe_buyt_detailed.csv'
extended_df.to_csv(output_filename, index=False, encoding='utf-8')

print(f'Dữ liệu chi tiết đã được lưu vào file {output_filename}. Tổng số tuyến: {len(extended_data)}')

# Hiển thị DataFrame (comment nếu không cần)
extended_df.head()

Đang scrape tuyến 01: Bến Thành- BX Chợ Lớn
Đang scrape tuyến 02: Bến Thành- BX Miền Tây
Đang scrape tuyến 03: Bến Thành- Thạnh Lộc
Đang scrape tuyến 04: Bến Thành- Cộng Hòa- An Sương
Đang scrape tuyến 05: Bến xe Chợ Lớn - Biên Hòa
Đang scrape tuyến 06: Bến xe Chợ Lớn- Đại học Nông Lâm
Đang scrape tuyến 07: Bến xe Chợ Lớn- Gò vấp
Đang scrape tuyến 08: Bến xe Quận 8- Đại học Quốc Gia
Đang scrape tuyến 09: Chợ Lớn - Bình Chánh - Hưng Long
Đang scrape tuyến 10: Đại học Quốc Gia- Bến xe Miền Tây
Đang scrape tuyến 11: Bến Thành- Đầm Sen
Đang scrape tuyến 12: Bến Thành - Thác Giang Điền
Đang scrape tuyến 13: Bến Thành- Bến xe Củ Chi
Đang scrape tuyến 14: Miền Đông- 3 tháng 2- Miền Tây
Đang scrape tuyến 15: Bến Phú Định- Đầm Sen
Đang scrape tuyến 16: Bến xe Chợ Lớn - Bến xe Tân Phú
Đang scrape tuyến 17: Bến Xe Chợ Lớn - ĐH Sài Gòn - KCX Tân Thuận
Đang scrape tuyến 18: Bến Thành - Chợ Hiệp Thành
Đang scrape tuyến 19: Bến Thành - Khu Chế Xuất Linh Trung - Đại Học Quốc Gia
Đang scrape tuyến 2

,Route_Number,Route_Name,URL,Operator,Operating_Type,Distance,Vehicle_Type,Operating_Hours,Ticket_Price,Number_of_Trips,Trip_Time,Frequency
0,01,Bến Thành- BX Chợ Lớn,https://xebuyt.net/ben-thanh-ben-xe-cho-lon.html,"Công ty Cổ phần Xe khách Sài Gòn, ĐT: (08) 38.441.224",Phổ thông - Có trợ giá,8.6 km,80 chỗ,05:00 - 20:30,"&nbsp- Vé lượt: 5,000 VNĐ\n &nbsp- Vé lượt HSSV: 2,000 VNĐ\n &nbsp- Vé tập: 112,500 VNĐ",240 chuyến/ngày,30 - 35 phút,6 - 10 phút
1,02,Bến Thành- BX Miền Tây,https://xebuyt.net/ben-thanh-ben-xe-mien-tay.html,"Công ty TNHH Vận tải Ngôi sao Sài Gòn, ĐT: (08) 38.642.764",Phổ thông - Có trợ giá,13.65 km,30 - 80 chỗ,04:45 - 18:30,"&nbsp- Vé lượt: 5,000 VNĐ\n &nbsp- Vé lượt HSSV: 2,000 VNĐ\n &nbsp- Vé tập: 112,500 VNĐ",180 chuyến/ngày,45 - 50 phút,8 - 12 phút
2,03,Bến Thành- Thạnh Lộc,https://xebuyt.net/ben-thanh-thanh-loc.html,"Công ty TNHH Vận tải Ngôi sao Sài Gòn, ĐT: (08) 38.642.764\nHợp tác xã vận tải 19/5, ĐT: (08) 37.130.711",Phổ thông - Có trợ giá,18.05 km,26 - 55 chỗ,04:15 - 20:45,"&nbsp- Vé lượt: 6,000 VNĐ\n &nbsp- Vé lượt HSSV: 2,000 VNĐ\n &nbsp- Vé tập: 135,000 VNĐ",312 chuyến/ngày,55 - 65 phút,4 - 12 phút
3,04,Bến Thành- Cộng Hòa- An Sương,https://xebuyt.net/ben-thanh-cong-hoa-an-suong.html,"Công ty TNHH Vận tải Ngôi sao Sài Gòn, ĐT: (08) 38.642.764\nHợp tác xã vận tải 19/5, ĐT: (08) 37.130.711",Phổ thông - Có trợ giá,16.6 km,30 - 55 chỗ,05:00 - 20:30,"&nbsp- Vé lượt: 5,000 VNĐ\n &nbsp- Vé lượt HSSV: 2,000 VNĐ\n &nbsp- Vé tập: 112,500 VNĐ",303 chuyến/ngày,50 - 65 phút,5 - 10 phút
4,05,Bến xe Chợ Lớn - Biên Hòa,https://xebuyt.net/ben-xe-cho-lon-bien-hoa.html,"Hợp tác xã vận tải xe buýt Quyết Thắng, ĐT: (08) 38.642.712\nHợp tác xã vận tải 19/5, ĐT: (08) 37.130.711\nHợp tác xã vận tải số 15, ĐT: (08) 62.820.071",Phổ thông - Không trợ giá,38 km,30 - 80 chỗ,04:50 - 17:50,,72 chuyến/ngày,60 - 95 phút,20 - 25 phút


# Lấy lộ trình

In [15]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import re

# Đọc file CSV đã tạo trước đó
csv_filename = 'tuyen_xe_buyt.csv'
df = pd.read_csv(csv_filename)

# Danh sách để lưu dữ liệu lộ trình (mỗi stop là một row)
route_stops_data = []

# Duyệt qua từng hàng trong DataFrame
for index, row in df.iterrows():
    route_number = row['Route_Number']
    route_name = row['Route_Name']
    url = row['URL']

    print(f"Đang scrape lộ trình tuyến {route_number}: {route_name}")

    try:
        # Gửi request
        response = requests.get(url)
        response.raise_for_status()

        # Parse HTML
        soup = BeautifulSoup(response.content, 'html.parser')

        # Tìm div lotrinh
        lotrinh_div = soup.find('div', id='lotrinh')
        if not lotrinh_div:
            print(f"Không tìm thấy div#lotrinh cho tuyến {route_number}")
            continue

        # Tìm tất cả accordion headers và contents
        accordion = lotrinh_div.find('div', id='accordion')
        if not accordion:
            print(f"Không tìm thấy accordion cho tuyến {route_number}")
            continue

        # Tìm tất cả h3 (headers cho hướng)
        headers = accordion.find_all('h3', class_='ui-accordion-header')

        for header in headers:
            # Lấy tên hướng (direction)
            direction = header.get_text(strip=True)

            # Tìm div content tương ứng (thường là sibling next div)
            content_div = header.find_next_sibling('div', class_='ui-accordion-content')
            if not content_div:
                continue

            # Tìm table trong content
            table = content_div.find('table')
            if not table:
                continue

            # Tìm tất cả rows trong tbody
            rows = table.find('tbody').find_all('tr', class_='rowStop')

            # Duyệt qua từng stop
            for row in rows:
                tds = row.find_all('td')
                if len(tds) < 2:
                    continue

                # Lấy thứ tự stop (A, 1, 2, ..., B)
                order_no_td = tds[0].find('div')
                stop_order = order_no_td.get_text(strip=True) if order_no_td else ''

                # Lấy tên stop
                stop_name_td = tds[1]
                stop_name = stop_name_td.get_text(strip=True)

                # Thêm vào danh sách
                route_stops_data.append({
                    'Route_Number': route_number,
                    'Route_Name': route_name,
                    'URL': url,
                    'Direction': direction,
                    'Stop_Order': stop_order,
                    'Stop_Name': stop_name
                })

    except Exception as e:
        print(f"Lỗi khi scrape lộ trình tuyến {route_number}: {e}")

    # Delay để tránh overload server
    time.sleep(1)

# Tạo DataFrame và lưu vào CSV mới
route_stops_df = pd.DataFrame(route_stops_data)
output_filename = 'tuyen_xe_buyt_lotrinh.csv'
route_stops_df.to_csv(output_filename, index=False, encoding='utf-8')

print(f'Dữ liệu lộ trình đã được lưu vào file {output_filename}. Tổng số stops: {len(route_stops_data)}')

# Hiển thị DataFrame (comment nếu không cần)
route_stops_df.head(10)

Đang scrape lộ trình tuyến 01: Bến Thành- BX Chợ Lớn
Đang scrape lộ trình tuyến 02: Bến Thành- BX Miền Tây
Đang scrape lộ trình tuyến 03: Bến Thành- Thạnh Lộc
Đang scrape lộ trình tuyến 04: Bến Thành- Cộng Hòa- An Sương
Đang scrape lộ trình tuyến 05: Bến xe Chợ Lớn - Biên Hòa
Đang scrape lộ trình tuyến 06: Bến xe Chợ Lớn- Đại học Nông Lâm
Đang scrape lộ trình tuyến 07: Bến xe Chợ Lớn- Gò vấp
Đang scrape lộ trình tuyến 08: Bến xe Quận 8- Đại học Quốc Gia
Đang scrape lộ trình tuyến 09: Chợ Lớn - Bình Chánh - Hưng Long
Đang scrape lộ trình tuyến 10: Đại học Quốc Gia- Bến xe Miền Tây
Đang scrape lộ trình tuyến 11: Bến Thành- Đầm Sen
Đang scrape lộ trình tuyến 12: Bến Thành - Thác Giang Điền
Đang scrape lộ trình tuyến 13: Bến Thành- Bến xe Củ Chi
Đang scrape lộ trình tuyến 14: Miền Đông- 3 tháng 2- Miền Tây
Đang scrape lộ trình tuyến 15: Bến Phú Định- Đầm Sen
Đang scrape lộ trình tuyến 16: Bến xe Chợ Lớn - Bến xe Tân Phú
Đang scrape lộ trình tuyến 17: Bến Xe Chợ Lớn - ĐH Sài Gòn - KCX Tân

""


In [16]:
import requests
from bs4 import BeautifulSoup
import html
import pandas as pd
import time
from pprint import pprint  # Để debug nếu cần

# Đọc file CSV đã tạo trước đó
csv_filename = 'tuyen_xe_buyt.csv'
try:
    df = pd.read_csv(csv_filename)
    print(f"Đã đọc thành công {len(df)} tuyến từ file {csv_filename}")
except FileNotFoundError:
    print(f"Không tìm thấy file {csv_filename}. Vui lòng chạy code scrape URL trước!")
    exit()

# Danh sách để lưu dữ liệu lộ trình (mỗi stop là một row)
route_stops_data = []

# Giới hạn số tuyến để test (bỏ comment nếu muốn scrape tất cả)
# df = df.head(5)  # Chỉ scrape 5 tuyến đầu để test

print(f"Bắt đầu scrape lộ trình cho {len(df)} tuyến...")

# Duyệt qua từng hàng trong DataFrame
for index, row in df.iterrows():
    route_number = row['Route_Number']
    route_name = row['Route_Name']
    url = row['URL']

    print(f"\n--- Đang scrape lộ trình tuyến {route_number}: {route_name} ---")
    print(f"URL: {url}")

    try:
        # Gửi yêu cầu HTTP để lấy nội dung trang web
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # Kiểm tra lỗi HTTP

        # Giải mã HTML để xử lý các ký tự đặc biệt
        html_content = html.unescape(response.text)

        # Sử dụng BeautifulSoup để phân tích HTML
        soup = BeautifulSoup(html_content, 'html.parser')
        print(f"✅ Đã parse HTML thành công. Độ dài content: {len(response.text)}")

        # Khởi tạo danh sách stops cho tuyến này
        stops_data = []

        # Biến để xác định hướng hiện tại
        current_direction = None

        # Duyệt qua các element h3 và table theo thứ tự
        for element in soup.find_all(['h3', 'table']):
            if element.name == 'h3':
                # Cập nhật hướng dựa trên tiêu đề (tìm các pattern phổ biến như "Đi [Điểm cuối]")
                header_text = element.get_text(strip=True)
                if "Đi" in header_text:
                    # Lấy phần sau "Đi " làm tên hướng
                    direction_name = header_text.replace("Đi ", "").strip()
                    current_direction = direction_name
                    print(f"  ✅ Phát hiện hướng mới: {current_direction}")
                else:
                    current_direction = None
            elif element.name == 'table' and current_direction:
                # Tìm tất cả các hàng trong bảng
                rows = element.find_all('tr')
                stop_count = 0
                for row in rows:
                    cols = row.find_all('td')
                    if len(cols) >= 2:
                        # Lấy thứ tự stop (từ cột 1, thường là <div> với A/1/2/.../B)
                        order_col = cols[0].find('div')
                        stop_order = order_col.get_text(strip=True) if order_col else str(stop_count + 1)

                        # Lấy tên stop từ cột 2
                        stop_name = cols[1].get_text(strip=True)

                        if stop_name and stop_name != '':  # Bỏ qua stop rỗng
                            # Thêm vào danh sách
                            stops_data.append({
                                'Route_Number': route_number,
                                'Route_Name': route_name,
                                'URL': url,
                                'Direction': current_direction,
                                'Stop_Order': stop_order,
                                'Stop_Name': stop_name
                            })
                            stop_count += 1
                            if stop_count <= 3:  # Chỉ print 3 stops đầu mỗi hướng để tránh spam
                                print(f"    Stop {stop_order}: {stop_name}")
                            if stop_count == 4:
                                print(f"    ... (và {len(rows) - stop_count + 1} stops khác)")

                print(f"  ✅ Thu thập {stop_count} stops cho hướng '{current_direction}'")

        # Thêm stops của tuyến này vào danh sách tổng
        route_stops_data.extend(stops_data)
        total_stops_this_route = len(stops_data)
        print(f"✅ Hoàn thành tuyến {route_number}: {total_stops_this_route} stops tổng cộng")

        # Debug: In preview stops nếu cần (comment nếu không muốn)
        # pprint(stops_data, indent=2)

    except requests.exceptions.RequestException as e:
        print(f"❌ Lỗi HTTP cho tuyến {route_number}: {e}")
    except Exception as e:
        print(f"❌ Lỗi khi xử lý dữ liệu tuyến {route_number}: {e}")
        import traceback
        traceback.print_exc()  # In chi tiết lỗi để debug

    # Delay để tránh overload server
    # time.sleep(2)

print(f"\n--- Kết thúc scrape ---")
print(f"Tổng số stops thu thập được: {len(route_stops_data)}")

if len(route_stops_data) == 0:
    print("❌ Không có dữ liệu nào! Kiểm tra log trên để debug.")
    print("Có thể một số tuyến không có lộ trình, hoặc HTML thay đổi.")

# Tạo DataFrame và lưu vào CSV mới
if len(route_stops_data) > 0:
    route_stops_df = pd.DataFrame(route_stops_data)
    output_filename = 'tuyen_xe_buyt_lotrinh.csv'
    route_stops_df.to_csv(output_filename, index=False, encoding='utf-8')

    print(f"✅ Dữ liệu lộ trình đã được lưu vào file {output_filename}")

    # Hiển thị preview DataFrame (top 20 rows)
    print("\n--- Preview DataFrame (top 20 rows) ---")
    print(route_stops_df.head(20).to_string(index=False))

    # Thống kê nhanh
    print(f"\n--- Thống kê ---")
    print(f"- Tổng tuyến có dữ liệu: {route_stops_df['Route_Number'].nunique()}")
    print(f"- Hướng phổ biến nhất: {route_stops_df['Direction'].value_counts().head(1)}")
else:
    print("❌ Không lưu file vì không có dữ liệu.")

Đã đọc thành công 140 tuyến từ file tuyen_xe_buyt.csv
Bắt đầu scrape lộ trình cho 140 tuyến...

--- Đang scrape lộ trình tuyến 01: Bến Thành- BX Chợ Lớn ---
URL: https://xebuyt.net/ben-thanh-ben-xe-cho-lon.html
✅ Đã parse HTML thành công. Độ dài content: 52658
  ✅ Phát hiện hướng mới: BX Chợ Lớn
    Stop A: Công Trường Mê Linh
    Stop 1: Bến Bạch Đằng
    Stop 2: Cục Hải Quan Thành Phố
    ... (và 26 stops khác)
  ✅ Thu thập 29 stops cho hướng 'BX Chợ Lớn'
  ✅ Phát hiện hướng mới: Bến Thành
    Stop A: Bến xe Chợ Lớn
    Stop 1: Chu Văn An
    Stop 2: Tháp Mười
    ... (và 32 stops khác)
  ✅ Thu thập 35 stops cho hướng 'Bến Thành'
  ✅ Phát hiện hướng mới: BX Chợ Lớn(Thứ 2, Thứ 3, Thứ 4, Thứ 5, Thứ 6, Thứ 7, Chủ Nhật)
  ✅ Thu thập 0 stops cho hướng 'BX Chợ Lớn(Thứ 2, Thứ 3, Thứ 4, Thứ 5, Thứ 6, Thứ 7, Chủ Nhật)'
    Stop 1: 05:00 - 05:35
    Stop 2: 05:10 - 05:45
    Stop 3: 05:20 - 05:55
    ... (và 117 stops khác)
  ✅ Thu thập 120 stops cho hướng 'BX Chợ Lớn(Thứ 2, Thứ 3, Thứ 4, Thứ 

In [17]:
import pandas as pd

# Thiết lập để hiển thị toàn bộ DataFrame (tắt giới hạn hàng và cột)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

# Hiển thị DataFrame

route_stops_df.head(300)

,Route_Number,Route_Name,URL,Direction,Stop_Order,Stop_Name
0,01,Bến Thành- BX Chợ Lớn,https://xebuyt.net/ben-thanh-ben-xe-cho-lon.html,BX Chợ Lớn,A,Công Trường Mê Linh
1,01,Bến Thành- BX Chợ Lớn,https://xebuyt.net/ben-thanh-ben-xe-cho-lon.html,BX Chợ Lớn,1,Bến Bạch Đằng
2,01,Bến Thành- BX Chợ Lớn,https://xebuyt.net/ben-thanh-ben-xe-cho-lon.html,BX Chợ Lớn,2,Cục Hải Quan Thành Phố
3,01,Bến Thành- BX Chợ Lớn,https://xebuyt.net/ben-thanh-ben-xe-cho-lon.html,BX Chợ Lớn,3,Chợ Cũ
4,01,Bến Thành- BX Chợ Lớn,https://xebuyt.net/ben-thanh-ben-xe-cho-lon.html,BX Chợ Lớn,4,Trường Cao Thắng
5,01,Bến Thành- BX Chợ Lớn,https://xebuyt.net/ben-thanh-ben-xe-cho-lon.html,BX Chợ Lớn,5,Công ty Đường sắt
6,01,Bến Thành- BX Chợ Lớn,https://xebuyt.net/ben-thanh-ben-xe-cho-lon.html,BX Chợ Lớn,6,Bến Thành B
7,01,Bến Thành- BX Chợ Lớn,https://xebuyt.net/ben-thanh-ben-xe-cho-lon.html,BX Chợ Lớn,7,Trường Ernst Thalmann
8,01,Bến Thành- BX Chợ Lớn,https://xebuyt.net/ben-thanh-ben-xe-cho-lon.html,BX Chợ Lớn,8,KTX Trần Hưng Đạo
9,01,Bến Thành- BX Chợ Lớn,https://xebuyt.net/ben-thanh-ben-xe-cho-lon.html,BX Chợ Lớn,9,Rạp Hưng Đạo


In [18]:
import requests
from bs4 import BeautifulSoup
import html
import pandas as pd
import time
import re  # Để parse thời gian

# Đọc file CSV đã tạo trước đó
csv_filename = 'tuyen_xe_buyt.csv'
try:
    df = pd.read_csv(csv_filename)
    print(f"Đã đọc thành công {len(df)} tuyến từ file {csv_filename}")
except FileNotFoundError:
    print(f"Không tìm thấy file {csv_filename}. Vui lòng chạy code scrape URL trước!")
    exit()

# Danh sách để lưu dữ liệu
route_stops_data = []      # Cho lộ trình (stops)
timetable_data = []        # Cho biểu đồ giờ (timetable)

# Giới hạn số tuyến để test (bỏ comment nếu muốn scrape tất cả)
df = df.head(5)  # Chỉ scrape 5 tuyến đầu để test nhanh
print(f"Test với {len(df)} tuyến đầu tiên...")

print(f"Bắt đầu scrape lộ trình + timetable cho {len(df)} tuyến...")

# Duyệt qua từng hàng trong DataFrame
for index, row in df.iterrows():
    route_number = row['Route_Number']
    route_name = row['Route_Name']
    url = row['URL']

    print(f"\n--- Đang scrape tuyến {route_number}: {route_name} ---")
    print(f"URL: {url}")

    try:
        # Gửi yêu cầu HTTP
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        response = requests.get(url, headers=headers)
        response.raise_for_status()

        # Giải mã HTML
        html_content = html.unescape(response.text)
        soup = BeautifulSoup(html_content, 'html.parser')
        print(f"✅ Đã parse HTML. Độ dài: {len(response.text)}")

        # PHẦN 1: EXTRACT LỘ TRÌNH (STOPS) - Giữ nguyên logic của bạn
        print("  --- Extract lộ trình ---")
        stops_this_route = []  # Stops tạm cho tuyến này
        current_direction = None

        # Duyệt h3 và table cho lộ trình (thường ở #lotrinh)
        lotrinh_div = soup.find('div', id='lotrinh') or soup  # Fallback nếu không có id
        for element in lotrinh_div.find_all(['h3', 'table']) if lotrinh_div else soup.find_all(['h3', 'table']):
            if element.name == 'h3':
                header_text = element.get_text(strip=True)
                if "Đi" in header_text:
                    current_direction = header_text.replace("Đi ", "").strip()
                    print(f"    Hướng: {current_direction}")
            elif element.name == 'table' and current_direction:
                rows = element.find_all('tr')
                stop_count = 0
                for row in rows:
                    cols = row.find_all('td')
                    if len(cols) >= 2:
                        # Thứ tự stop
                        order_col = cols[0].find('div')
                        stop_order = order_col.get_text(strip=True) if order_col else str(stop_count + 1)

                        # Tên stop
                        stop_name = cols[1].get_text(strip=True)

                        if stop_name:
                            stops_this_route.append({
                                'Route_Number': route_number,
                                'Route_Name': route_name,
                                'URL': url,
                                'Direction': current_direction,
                                'Stop_Order': stop_order,
                                'Stop_Name': stop_name
                            })
                            stop_count += 1
                            if stop_count <= 2:  # Preview 2 stops
                                print(f"      {stop_order}: {stop_name}")

                print(f"    ✅ {stop_count} stops cho '{current_direction}'")

        route_stops_data.extend(stops_this_route)
        print(f"  ✅ Tổng stops tuyến này: {len(stops_this_route)}")

        # PHẦN 2: EXTRACT BIỂU ĐỒ GIỜ (TIMETABLE) - Parse table có thời gian
        print("  --- Extract timetable ---")
        timetable_this_route = []  # Timetable tạm cho tuyến này

        # Tìm tất cả table (timetable thường là table sau lộ trình, có text giờ)
        all_tables = soup.find_all('table')
        for table in all_tables:
            rows = table.find_all('tr')
            for row in rows:
                cols = row.find_all('td')
                if len(cols) >= 4:  # Giả sử timetable có >=4 cột: Stop_Order, Stop_Name, ..., Thời gian
                    # Lấy Stop_Order (cột 1, thường số)
                    stop_order = cols[0].get_text(strip=True) if cols[0] else ''

                    # Lấy Stop_Name (cột 2)
                    stop_name = cols[1].get_text(strip=True) if len(cols) > 1 else ''

                    # Tìm thời gian ở các cột sau (pattern HH:MM-HH:MM hoặc HH:MM)
                    time_text = ''
                    for col in cols[2:]:  # Từ cột 3 trở đi
                        col_text = col.get_text(strip=True)
                        # Regex match thời gian (e.g., "05:00-05:35", "05:00")
                        time_match = re.search(r'\d{2}:\d{2}(-\d{2}:\d{2})?', col_text)
                        if time_match:
                            time_text = time_match.group(0)
                            break

                    if stop_name and time_text:  # Chỉ lấy nếu có stop và thời gian
                        # Direction fallback (dùng hướng từ lộ trình gần nhất, hoặc default)
                        direction = current_direction or "Không xác định"

                        timetable_this_route.append({
                            'Route_Number': route_number,
                            'Route_Name': route_name,
                            'URL': url,
                            'Direction': direction,
                            'Stop_Order': stop_order,
                            'Stop_Name': stop_name,
                            'Departure_Time': time_text  # e.g., "05:00-05:35"
                        })

                        print(f"      {stop_order} - {stop_name}: {time_text}")

        timetable_data.extend(timetable_this_route)
        print(f"  ✅ Tổng timetable rows tuyến này: {len(timetable_this_route)}")

    except requests.exceptions.RequestException as e:
        print(f"❌ Lỗi HTTP: {e}")
    except Exception as e:
        print(f"❌ Lỗi xử lý: {e}")
        import traceback
        traceback.print_exc()

    time.sleep(2)  # Delay

print(f"\n--- Kết thúc scrape ---")
print(f"- Tổng stops (lộ trình): {len(route_stops_data)}")
print(f"- Tổng rows (timetable): {len(timetable_data)}")

# Lưu file lộ trình (chỉ stops)
if route_stops_data:
    lotrinh_df = pd.DataFrame(route_stops_data)
    lotrinh_df.to_csv('tuyen_xe_buyt_lotrinh.csv', index=False, encoding='utf-8')
    print(f"✅ Lưu lộ trình vào tuyen_xe_buyt_lotrinh.csv")
    print(lotrinh_df.head(10).to_string(index=False))  # Preview

# Lưu file timetable (chỉ giờ)
if timetable_data:
    timetable_df = pd.DataFrame(timetable_data)
    timetable_df.to_csv('tuyen_xe_buyt_timetable.csv', index=False, encoding='utf-8')
    print(f"✅ Lưu timetable vào tuyen_xe_buyt_timetable.csv")
    print(timetable_df.head(10).to_string(index=False))  # Preview
else:
    print("❌ Không có timetable data (có thể table giờ không match pattern).")

Đã đọc thành công 140 tuyến từ file tuyen_xe_buyt.csv
Test với 5 tuyến đầu tiên...
Bắt đầu scrape lộ trình + timetable cho 5 tuyến...

--- Đang scrape tuyến 01: Bến Thành- BX Chợ Lớn ---
URL: https://xebuyt.net/ben-thanh-ben-xe-cho-lon.html
✅ Đã parse HTML. Độ dài: 52658
  --- Extract lộ trình ---
    Hướng: BX Chợ Lớn
      A: Công Trường Mê Linh
      1: Bến Bạch Đằng
    ✅ 29 stops cho 'BX Chợ Lớn'
    Hướng: Bến Thành
      A: Bến xe Chợ Lớn
      1: Chu Văn An
    ✅ 35 stops cho 'Bến Thành'
  ✅ Tổng stops tuyến này: 64
  --- Extract timetable ---
  ✅ Tổng timetable rows tuyến này: 0

--- Đang scrape tuyến 02: Bến Thành- BX Miền Tây ---
URL: https://xebuyt.net/ben-thanh-ben-xe-mien-tay.html
✅ Đã parse HTML. Độ dài: 52394
  --- Extract lộ trình ---
    Hướng: BX Miền Tây
      A: Công Trường Mê Linh
      1: Bến Bạch Đằng
    ✅ 43 stops cho 'BX Miền Tây'
    Hướng: Bến Thành
      A: Bến xe Miền Tây
      1: 49-51 (317), Kinh Dương Vương
    ✅ 37 stops cho 'Bến Thành'
  ✅ Tổng stops

In [19]:
import requests
from bs4 import BeautifulSoup
import html
import pandas as pd
import time

# Đọc file CSV đã tạo trước đó
csv_filename = 'tuyen_xe_buyt.csv'
try:
    df = pd.read_csv(csv_filename)
    print(f"Đã đọc thành công {len(df)} tuyến từ file {csv_filename}")
except FileNotFoundError:
    print(f"Không tìm thấy file {csv_filename}. Vui lòng chạy code scrape URL trước!")
    exit()

# Danh sách để lưu dữ liệu lộ trình (mỗi stop là một row)
route_stops_data = []

# Giới hạn số tuyến để test (bỏ comment nếu muốn scrape tất cả)
df = df.head(5)  # Chỉ scrape 5 tuyến đầu để test nhanh
print(f"Test với {len(df)} tuyến đầu tiên...")

print(f"Bắt đầu scrape lộ trình cho {len(df)} tuyến...")

# Duyệt qua từng hàng trong DataFrame
for index, row in df.iterrows():
    route_number = row['Route_Number']
    route_name = row['Route_Name']
    url = row['URL']

    print(f"\n--- Đang scrape lộ trình tuyến {route_number}: {route_name} ---")
    print(f"URL: {url}")

    try:
        # Gửi request với headers để giả lập browser
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        resp = requests.get(url, headers=headers)
        resp.raise_for_status()  # Kiểm tra lỗi HTTP

        # Giải mã HTML (tránh ký tự &nbsp; hoặc &amp;)
        html_content = html.unescape(resp.text)
        soup = BeautifulSoup(html_content, "html.parser")
        print(f"✅ Đã parse HTML thành công. Độ dài content: {len(resp.text)}")

        # Khởi tạo routes cho tuyến này (dynamic, không hardcoded)
        routes_this_line = {}  # {direction: [stops]}

        current_direction = None

        # Duyệt toàn bộ h3 và table theo thứ tự (như code của bạn, nhưng generalize)
        for element in soup.find_all(["h3", "table"]):
            if element.name == "h3":
                text = element.get_text(strip=True)
                # Generalize: Bất kỳ h3 nào chứa "Đi " là hướng mới
                if "Đi " in text:
                    current_direction = text.strip()  # e.g., "Đi BX Chợ Lớn"
                    routes_this_line[current_direction] = []  # Khởi tạo list stops
                    print(f"  ✅ Phát hiện hướng mới: {current_direction}")
                else:
                    current_direction = None
            elif element.name == "table" and current_direction:
                # Duyệt qua các row với class="rowStop" (như code của bạn)
                stop_count = 0
                for row in element.find_all("tr", class_="rowStop"):
                    # Lấy Stop_Order từ td đầu tiên (class="orderNo" > div)
                    order_col = row.find("td", class_="orderNo")
                    if order_col:
                        order_div = order_col.find("div")
                        stop_order = order_div.get_text(strip=True) if order_div else str(stop_count + 1)
                    else:
                        stop_order = str(stop_count + 1)

                    # Lấy tên stop từ td class="stopName" (như code của bạn)
                    stop_col = row.find("td", class_="stopName")
                    if stop_col:
                        stop_name = stop_col.get_text(strip=True)
                        if stop_name:  # Bỏ qua stop rỗng
                            routes_this_line[current_direction].append({
                                'Stop_Order': stop_order,
                                'Stop_Name': stop_name
                            })
                            stop_count += 1
                            if stop_count <= 3:  # Preview 3 stops đầu
                                print(f"    {stop_order}: {stop_name}")
                            if stop_count == 4:
                                print(f"    ... (và {len(element.find_all('tr', class_='rowStop')) - stop_count} stops khác)")

                print(f"  ✅ Thu thập {stop_count} stops cho '{current_direction}'")

        # Thêm stops của tuyến này vào danh sách tổng (flat structure cho CSV)
        total_stops_this_route = 0
        for direction, stops in routes_this_line.items():
            for stop in stops:
                route_stops_data.append({
                    'Route_Number': route_number,
                    'Route_Name': route_name,
                    'URL': url,
                    'Direction': direction,
                    'Stop_Order': stop['Stop_Order'],
                    'Stop_Name': stop['Stop_Name']
                })
                total_stops_this_route += 1

        print(f"✅ Hoàn thành tuyến {route_number}: {total_stops_this_route} stops tổng cộng (từ {len(routes_this_line)} hướng)")

    except requests.exceptions.RequestException as e:
        print(f"❌ Lỗi HTTP cho tuyến {route_number}: {e}")
    except Exception as e:
        print(f"❌ Lỗi xử lý dữ liệu tuyến {route_number}: {e}")
        import traceback
        traceback.print_exc()  # In chi tiết lỗi để debug

    # Delay để tránh overload server
    time.sleep(2)

print(f"\n--- Kết thúc scrape ---")
print(f"Tổng số stops thu thập được: {len(route_stops_data)}")

if len(route_stops_data) == 0:
    print("❌ Không có dữ liệu nào! Kiểm tra log trên để debug.")
    print("💡 Có thể một số tuyến không có h3 'Đi ', hoặc HTML thay đổi. Thử Selenium nếu cần.")

# Tạo DataFrame và lưu vào CSV mới
if len(route_stops_data) > 0:
    route_stops_df = pd.DataFrame(route_stops_data)
    output_filename = 'tuyen_xe_buyt_lotrinh.csv'
    route_stops_df.to_csv(output_filename, index=False, encoding='utf-8')

    print(f"✅ Dữ liệu lộ trình đã được lưu vào file {output_filename}")

    # Hiển thị DataFrame (top 20 rows để xem)
    print("\n--- Preview DataFrame (top 20 rows) ---")
    print(route_stops_df.head(20).to_string(index=False))

    # Thống kê nhanh
    print(f"\n--- Thống kê ---")
    print(f"- Tổng tuyến có dữ liệu: {route_stops_df['Route_Number'].nunique()}")
    print(f"- Số hướng: {route_stops_df['Direction'].nunique()}")
    print(f"- Stops trung bình/tuyến: {len(route_stops_df) / route_stops_df['Route_Number'].nunique():.1f}")
    print(f"- Hướng phổ biến nhất: {route_stops_df['Direction'].value_counts().head(1)}")
else:
    print("❌ Không lưu file vì không có dữ liệu.")

Đã đọc thành công 140 tuyến từ file tuyen_xe_buyt.csv
Test với 5 tuyến đầu tiên...
Bắt đầu scrape lộ trình cho 5 tuyến...

--- Đang scrape lộ trình tuyến 01: Bến Thành- BX Chợ Lớn ---
URL: https://xebuyt.net/ben-thanh-ben-xe-cho-lon.html
✅ Đã parse HTML thành công. Độ dài content: 52658
  ✅ Phát hiện hướng mới: Đi BX Chợ Lớn
    A: Công Trường Mê Linh
    1: Bến Bạch Đằng
    2: Cục Hải Quan Thành Phố
    ... (và 25 stops khác)
  ✅ Thu thập 29 stops cho 'Đi BX Chợ Lớn'
  ✅ Phát hiện hướng mới: Đi Bến Thành
    A: Bến xe Chợ Lớn
    1: Chu Văn An
    2: Tháp Mười
    ... (và 31 stops khác)
  ✅ Thu thập 35 stops cho 'Đi Bến Thành'
  ✅ Phát hiện hướng mới: Đi BX Chợ Lớn(Thứ 2, Thứ 3, Thứ 4, Thứ 5, Thứ 6, Thứ 7, Chủ Nhật)
  ✅ Thu thập 0 stops cho 'Đi BX Chợ Lớn(Thứ 2, Thứ 3, Thứ 4, Thứ 5, Thứ 6, Thứ 7, Chủ Nhật)'
  ✅ Thu thập 0 stops cho 'Đi BX Chợ Lớn(Thứ 2, Thứ 3, Thứ 4, Thứ 5, Thứ 6, Thứ 7, Chủ Nhật)'
  ✅ Phát hiện hướng mới: Đi Bến Thành(Thứ 2, Thứ 3, Thứ 4, Thứ 5, Thứ 6, Thứ 7, Chủ Nh

In [20]:
route_stops_df.head(100)

,Route_Number,Route_Name,URL,Direction,Stop_Order,Stop_Name
0,01,Bến Thành- BX Chợ Lớn,https://xebuyt.net/ben-thanh-ben-xe-cho-lon.html,Đi BX Chợ Lớn,A,Công Trường Mê Linh
1,01,Bến Thành- BX Chợ Lớn,https://xebuyt.net/ben-thanh-ben-xe-cho-lon.html,Đi BX Chợ Lớn,1,Bến Bạch Đằng
2,01,Bến Thành- BX Chợ Lớn,https://xebuyt.net/ben-thanh-ben-xe-cho-lon.html,Đi BX Chợ Lớn,2,Cục Hải Quan Thành Phố
3,01,Bến Thành- BX Chợ Lớn,https://xebuyt.net/ben-thanh-ben-xe-cho-lon.html,Đi BX Chợ Lớn,3,Chợ Cũ
4,01,Bến Thành- BX Chợ Lớn,https://xebuyt.net/ben-thanh-ben-xe-cho-lon.html,Đi BX Chợ Lớn,4,Trường Cao Thắng
5,01,Bến Thành- BX Chợ Lớn,https://xebuyt.net/ben-thanh-ben-xe-cho-lon.html,Đi BX Chợ Lớn,5,Công ty Đường sắt
6,01,Bến Thành- BX Chợ Lớn,https://xebuyt.net/ben-thanh-ben-xe-cho-lon.html,Đi BX Chợ Lớn,6,Bến Thành B
7,01,Bến Thành- BX Chợ Lớn,https://xebuyt.net/ben-thanh-ben-xe-cho-lon.html,Đi BX Chợ Lớn,7,Trường Ernst Thalmann
8,01,Bến Thành- BX Chợ Lớn,https://xebuyt.net/ben-thanh-ben-xe-cho-lon.html,Đi BX Chợ Lớn,8,KTX Trần Hưng Đạo
9,01,Bến Thành- BX Chợ Lớn,https://xebuyt.net/ben-thanh-ben-xe-cho-lon.html,Đi BX Chợ Lớn,9,Rạp Hưng Đạo


# Lấy lộ trình - fixed

# Lấy biểu đồ giờ